<a href="https://colab.research.google.com/github/FreshMag/omr-img2midi/blob/master/AE2E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Augmented End-to-End - Setup
In this notebook we will use in practice all the code written for the Augmented End-to-End part of the project.

## Getting the required code and models

We need first to clone the repository with the code we need plus the original model built and trained by Carlo-Zaragoza on the PrImuS dataset

In [ ]:
!git clone https://github.com/FreshMag/omr-img2midi.git
%cd omr-img2midi
!mkdir models
%cd models
!wget https://grfia.dlsi.ua.es/primus/models/PrIMuS/Semantic-Model.zip
!unzip Semantic-Model.zip -d semantic
!rm Semantic-Model.zip
%cd ../..
!mkdir outputs
%cd omr-img2midi

## Getting the Java tools for conversion: `semantic → MIDI/MEI`

This is necessary to view our produced symbols in other more readable formats. This tool was developed by Carlo-Zaragoza and converts a file written in semantic format to MEI or MIDI format.

In [ ]:
!wget https://grfia.dlsi.ua.es/primus/primus_converter.tgz
!tar xzvf primus_converter.tgz
!mv primus_conversor/omr-3.0-SNAPSHOT.jar ./converter/omr-3.0-SNAPSHOT.jar
!rm -rf ./primus_conversor
!rm ./primus_converter.tgz

## Installing required libraries

All the libraries specified by ``pyproject.toml`` are installed + the library ``verovio`` used for rendering MEI files into SVG.

In [ ]:
!pip install "/content/omr-img2midi"
!pip install verovio

import sys
sys.path.append('/content/omr-img2midi/src')

## Import of python libraries and setup

Some of the used libraries are imported and the main paths are defined.

In [ ]:
from semantic.sheet import EncodedSheet
from semantic.end2end.ctc_predict import CTC
import verovio
import cv2

base_path = "/content/omr-img2midi/"
model_path = base_path + "models/semantic/semantic_model.meta"
input_image_path = "/content/"
vocabulary_path = base_path + "data/vocabulary_semantic.txt"

outputs_path = "/content/outputs/"


## Utility Functions

The following functions are used in the rest of the code. They are trivial.

In [ ]:
from IPython.display import SVG, display, Markdown
import os

def get_filename_with_ex(filename, extension):
  return filename[:filename.rfind(".")] + "." + extension

def show_svg_file(filename):
  display(SVG(filename=filename))

def show_markdown(markdown_str):
  display(Markdown(markdown_str))

def mei2svg(input_mei_file, output_svg_file):
  tk = verovio.toolkit()
  tk.loadFile(input_mei_file)

  # Setting of verovio layout options
  options = {
      "scale": 40,
      "adjustPageHeight": True,
      "backgroundColor": "white"  # White background color
  }

  tk.setOptions(options)
  tk.renderToSVGFile(output_svg_file, 1)


## Getting test image

We need now to get a test image to apply our algorithm

In [ ]:
%cd /content
!gdown 19hz4krss-w-q3PEiMu8KeB2PUiB0HXE9

## Input displayed

In [ ]:
from google.colab.patches import cv2_imshow

# Set the image name here
image_name = "test_end2end.jpg"

final_input_path = input_image_path + image_name
photo = cv2.imread(final_input_path, cv2.IMREAD_COLOR)

cv2_imshow(cv2.resize(photo, (photo.shape[1] // 4, photo.shape[0] // 4)))

# Application of the algorithms

In the following cells we now apply all the algorithms implemented in this project.

## First part - Scanning

Here we scan the photo, using the `img2doc.scan` function. The produced image is a scanned-like image.

In [ ]:
import core.img2doc.scanner as img2doc
scanned = img2doc.scan(photo)

cv2_imshow(cv2.resize(scanned, (scanned.shape[1] // 4, scanned.shape[0] // 4)))

## Second part - Segmentation

Here we divide the scanned image in segments using the ``doc2segments.segment_doc`` function. The result is a list of sub-images, each part of the input of the model.



In [ ]:
from core.doc2segments.segmentation import doc2segments

segments = doc2segments.segment_doc(scanned)

cv2_imshow(segments[0])

## Usage of the model

We can now feed our produced segments into the model. For this, a utility function is been used: ``end2end_recognition``. This function takes a list of segments, the object encapsulating the model and the path to the vocabulary used by the model.

In [ ]:
from semantic.end2end.ctc_predict import CTC
from util import end2end_recognition

model = CTC(model_file_path=model_path)
sheet, subsheets = end2end_recognition(segments, model=model, vocabulary_path=vocabulary_path)

## Output

The result of the recognition is an instance of the class ``EncodedSheet``. This class contains several utility methods that we can use to get the result in multiple formats.

### Symbols printed
Here we print a list of the symbols produced by the model.

In [ ]:
sheet.print_symbols()

### Output to file

Using the method inside ``EncodedSheet``, we output the result of the prediction inside a file with  ``.semantic`` extension.

In [ ]:
semantic_filename = get_filename_with_ex(image_name, "semantic")
semantic_path = outputs_path + semantic_filename

sheet.clean(symbol="tie")
sheet.write_to_file(semantic_path, separator="\t")

# Showing the results

We can now convert the symbols we found onto different formats, for example the MEI format.

## Generation of the MEI file

Using the tool we previously downloaded, we can convert the file with the `.semantic` extension into a file with `.mei` extension.


In [ ]:
mei_filename = get_filename_with_ex(image_name, "mei")
output_mei_path = outputs_path + mei_filename

print(f"Converting to {output_mei_path}")
%cd /content/omr-img2midi
!./converter.sh {semantic_path} {output_mei_path}

## Rendering MEI file into SVG

Using the `verovio` library, we now convert the `.mei` file into a `.svg` file.

In [ ]:
output_svg_path = outputs_path + get_filename_with_ex(image_name, "svg")

# utility function that converts MEI file into SVG file
mei2svg(output_mei_path, output_svg_path)


## Show the result

Here we show the produced SVG file.

In [ ]:
from google.colab.patches import cv2_imshow

show_markdown("## Generated Input")
show_svg_file(output_svg_path)